In [44]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score, mean_squared_error as MSE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN 

In [45]:
data = pd.read_csv("Titanic.csv")

### Data Exploration

In [46]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


- The age range looks natural without outliers 

In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


- as we notice that the column Cabin has many null values compared with other columns

In [49]:
data.shape

(891, 12)

In [50]:
print(data.duplicated().sum())

0


* as seen there is no duplicated values

In [51]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Data Cleaning

In [52]:
data[data["Cabin"].isna()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


- At First we will drop the PassengerId, Name because they don't have correlation with the target variable 
- The Cabin column also will we be dropped because it contain alot of nulls and don't have correlation with the target variable 

In [53]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
data[data["Embarked"].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [55]:
data[data["Fare"] == 80.0]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


- we can't predict the best value to impute the 'Embarked', 'Age' and 'Cabin' missing values

In [56]:
data.dropna(subset=["Age"],inplace=True)
data.dropna(subset=["Embarked"], inplace=True)
data.dropna(subset=["Cabin"], inplace=True)

In [57]:
print(data["Embarked"].isna().sum())
print(data["Age"].isna().sum())
print(data["Cabin"].isna().sum())

0
0
0


In [58]:
data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### Data Transformation

- now we want to convert columns data types to number types

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  183 non-null    int64  
 1   Survived     183 non-null    int64  
 2   Pclass       183 non-null    int64  
 3   Name         183 non-null    object 
 4   Sex          183 non-null    object 
 5   Age          183 non-null    float64
 6   SibSp        183 non-null    int64  
 7   Parch        183 non-null    int64  
 8   Ticket       183 non-null    object 
 9   Fare         183 non-null    float64
 10  Cabin        183 non-null    object 
 11  Embarked     183 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 18.6+ KB


* The Age column can't be float, instead we will convert it to integer

In [60]:
data["Age"] = data["Age"].astype('int')

In [61]:
print(data["Name"].str.lower().value_counts())

Name
cumings, mrs. john bradley (florence briggs thayer)     1
appleton, mrs. edward dale (charlotte lamson)           1
silvey, mrs. william baird (alice munger)               1
thayer, mrs. john borland (marian longstreth morris)    1
ross, mr. john hugo                                     1
                                                       ..
ball, mrs. (ada e hall)                                 1
hippach, miss. jean gertrude                            1
partner, mr. austen                                     1
graham, mr. george edward                               1
behr, mr. karl howell                                   1
Name: count, Length: 183, dtype: int64


In [62]:
print(f"The number of unique values in Passenger ID column : {data['PassengerId'].value_counts().sum()}")
print(f"The number of unique values in Ticket column : {data['Ticket'].value_counts().sum()}")
print(f"The number of unique values in Name column : {data['Name'].value_counts().sum()}")

The number of unique values in Passenger ID column : 183
The number of unique values in Ticket column : 183
The number of unique values in Name column : 183


* Now we ensured that all rows are unique and there is no repeated values

In [63]:
print(f"The number of unique values in Cabin column : {data['Cabin'].value_counts().sum()}")

The number of unique values in Cabin column : 183


### Feature Engineering 

In [ ]:
# plt.boxplot(x=data)
# plt.show()

* The numeric columns are not standardized

In [65]:
data.drop(columns=["Name"], inplace=True)

* The name column is dropped because it doesn't affect the model training

In [66]:
le = LabelEncoder()

data["Sex"] = le.fit_transform(data["Sex"])

In [67]:
data["Embarked"] = le.fit_transform(data["Embarked"])
data["Cabin"] = le.fit_transform(data["Cabin"])
data["Ticket"] =le.fit_transform(data["Ticket"])

In [68]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

data.columns

scaler.fit(data[["Age", "Ticket", "Fare", "Cabin"]])

data[["Age", "Ticket", "Fare", "Cabin"]] = scaler.transform(data[["Age", "Ticket", "Fare", "Cabin"]])

### Data Splitting

In [69]:
X = data[["Embarked", "Sex", "Cabin", "Ticket", "PassengerId", "Age", "SibSp", "Parch", "Fare", "Pclass"]]
y = data["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, stratify=y, random_state=1)

### Model Building

In [70]:
dtc = DecisionTreeClassifier(max_depth=3100,min_samples_leaf=0.15,min_samples_split=4)

dtc.fit(X_train, y_train)

y_train_pred = dtc.predict(X_train)
print(accuracy_score(y_train, y_train_pred))

y_pred = dtc.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7890625
0.7636363636363637


In [71]:
MSE_CV = - cross_val_score(dtc,X_train, y_train, cv = 10, scoring="neg_mean_squared_error",n_jobs=-1)

dtc.fit(X_train, y_train)

y_train_pred = dtc.predict(X_train)
y_pred_test = dtc.predict(X_test)

print("CV MSE : {:.2f}".format(MSE_CV.mean()))

CV MSE : 0.26


In [73]:
estimators = list(range(150,1500,30))
depths = list(range(2,10))
samples_splits = list(range(2,50))
samples_leaf = list(range(2,50))

index = 0

train_accuracies = []
test_accuracies= []

# for item1 in estimators:
#     for item2 in depths:
#         for item3 in samples_splits:
#             for item4 in samples_leaf:
# index +=1
rfc = RandomForestClassifier(n_estimators=500, max_depth=3, min_samples_split=5, min_samples_leaf=4)

rfc.fit(X_train, y_train)

y_train_pred = rfc.predict(X_train)
y_test_pred = rfc.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# train_accuracies.append(train_accuracy)
# test_accuracies.append(test_accuracy)

# print(np.mean(train_accuracies))

print("{:.2f}".format(train_accuracy))
print("{:.2f}".format(test_accuracy))

0.88
0.78
